## 데이터 전처리 및 통합

a. 데이터 통합

두 CSV 파일(예: 청소년 상담 사례 6,500건과 전처리된 270건의 상담 사례)을 하나의 데이터셋으로 합칩니다.

이때, 열의 형식(예, "prompt"와 "response" 또는 "question"과 "answer")이 동일한지 확인하고, 필요에 따라 통일합니다.

b. 추가 정보(카테고리, 시간 등) 확인

만약 상담 내용에 주제(예: 친구 문제, 가족 문제, 학교폭력 등)나 기타 기준이 있다면, 이를 하나의 열(예: "category")로 명시해 두면 나중에 분할 시에 stratified sampling(층화 샘플링)을 할 수 있어 데이터 분포를 균일하게 유지할 수 있습니다.

## 청소년 고민 순위 통계 데이터 먼저 전처리

In [3]:
import pandas as pd
df = pd.read_csv('/content/ky_yngbgs_worry_rank_info_202012 (1).csv')
df

,sn,base_ym,search_rank_nm,srchwrd_nm,views_co,pt_value
0,1,201901,1,친구,69,24.21
1,2,201901,2,전학,20,7.02
2,3,201901,3,공부,15,5.26
3,4,201901,4,숨쉬기,13,4.56
4,5,201901,5,학원,11,3.86
...,...,...,...,...,...,...
960,961,202011,42,질투,5,0.89
961,962,202011,43,시험,5,0.89
962,963,202011,44,인생,5,0.89
963,964,202011,45,사춘기,4,0.71


In [7]:
import pandas as pd

# 1. CSV 파일 로드 (파일 경로는 상황에 맞게 조정)
stat_file = "/content/ky_yngbgs_worry_rank_info_202012 (1).csv"
df_stat = pd.read_csv(stat_file)

# 2. 데이터 확인
print("통계 데이터 shape:", df_stat.shape)
print(df_stat)

# 3. 데이터 집계
# 여기서는 각 고민 주제(srchwrd_nm)에 대해 pt_value의 평균을 계산하는 예시
# (필요에 따라 합계(sum)나 최대값(max) 등으로 변경 가능합니다.)
df_grouped = df_stat.groupby("srchwrd_nm")["pt_value"].mean().reset_index()
df_grouped = df_grouped.sort_values(by="pt_value", ascending=False)

print("\n각 고민 주제별 평균 pt_value:")
print(df_grouped)

# 4. 매핑 딕셔너리 생성
# 예시: {"친구": 24.21, "전학": 7.02, ...}
stat_mapping = dict(zip(df_grouped["srchwrd_nm"], df_grouped["pt_value"]))
print("\n통계 기반 매핑 딕셔너리:")
print(stat_mapping)

# 5. 만약 여러 통계 수치를 함께 고려하고 싶다면 views_co나 다른 값을 사용해 가중치를 조합하는 것도 가능

통계 데이터 shape: (965, 6)
      sn  base_ym  search_rank_nm srchwrd_nm  views_co  pt_value
0      1   201901               1         친구        69     24.21
1      2   201901               2         전학        20      7.02
2      3   201901               3         공부        15      5.26
3      4   201901               4        숨쉬기        13      4.56
4      5   201901               5         학원        11      3.86
..   ...      ...             ...        ...       ...       ...
960  961   202011              42         질투         5      0.89
961  962   202011              43         시험         5      0.89
962  963   202011              44         인생         5      0.89
963  964   202011              45        사춘기         4      0.71
964  965   202011              46          키         4      0.71

[965 rows x 6 columns]

각 고민 주제별 평균 pt_value:
    srchwrd_nm   pt_value
177         친구  19.389130
146         전학   5.969130
55          배정   4.760000
191         학교   4.345000
106         엄마   4.2

## 위 활용 예제

In [23]:
# 통게 데이터를 기반으로 우선순위 정하고, 상담 텍스트에 포함된 키워드를 찾아내어 적절한 주제로 분류할 수 있음
def assign_category_using_stat(prompt_text, stat_mapping):
    # stat_mapping의 키(고민 주제)들을 중요도(가중치) 기준 내림차순 정렬
    sorted_keywords = sorted(stat_mapping.items(), key=lambda x: x[1], reverse=True)
    for keyword, weight in sorted_keywords:
        if keyword in prompt_text:
            return keyword  # 필요에 따라 "키워드 문제" 형식으로 가공 가능
    return "기타"

# 예시 상담 문장
example_prompt = "친구와 갈등이 생겨 너무 고민이에요. 어떻게 해야 할지 모르겠어요."
assigned_category = assign_category_using_stat(example_prompt, stat_mapping)
print("할당된 카테고리:", assigned_category)

할당된 카테고리: 친구


In [24]:
# 통계 데이터 로드 및 확인
stat_file = "/content/ky_yngbgs_worry_rank_info_202012 (1).csv"
df_stat = pd.read_csv(stat_file)
print("통계 데이터 shape:", df_stat.shape)
print(df_stat.head())

# 그룹화: 각 고민 주제별 평균 pt_value 계산
df_grouped = df_stat.groupby("srchwrd_nm")["pt_value"].mean().reset_index()
df_grouped = df_grouped.sort_values(by="pt_value", ascending=False)
print(df_grouped.head(10))

# 매핑 딕셔너리 생성
stat_mapping = dict(zip(df_grouped["srchwrd_nm"], df_grouped["pt_value"]))
print("통계 기반 매핑 딕셔너리:")
print(stat_mapping)

통계 데이터 shape: (965, 6)
   sn  base_ym  search_rank_nm srchwrd_nm  views_co  pt_value
0   1   201901               1         친구        69     24.21
1   2   201901               2         전학        20      7.02
2   3   201901               3         공부        15      5.26
3   4   201901               4        숨쉬기        13      4.56
4   5   201901               5         학원        11      3.86
    srchwrd_nm   pt_value
177         친구  19.389130
146         전학   5.969130
55          배정   4.760000
191         학교   4.345000
106         엄마   4.286957
155       주휴수당   3.643333
139         자퇴   3.522609
21          공부   3.392174
23          관계   3.383043
72          생리   3.303478
통계 기반 매핑 딕셔너리:
{'친구': 19.389130434782608, '전학': 5.969130434782608, '배정': 4.760000000000001, '학교': 4.345000000000001, '엄마': 4.28695652173913, '주휴수당': 3.643333333333333, '자퇴': 3.5226086956521736, '공부': 3.3921739130434783, '관계': 3.38304347826087, '생리': 3.303478260869565, '가출': 3.18, '상담': 2.9466666666666668, '개학': 2.9083

In [25]:
def assign_category(prompt_text, stat_mapping):
    sorted_keywords = sorted(stat_mapping.items(), key=lambda x: x[1], reverse=True)
    for keyword, _ in sorted_keywords:
        if keyword in prompt_text:
            return keyword
    return "기타"

# 통합된 데이터셋에 대해 "extracted_category" 열 생성
df_combined["extracted_category"] = df_combined["prompt"].apply(lambda x: assign_category(x, stat_mapping))
# 필요에 따라 df_combined를 다시 저장하여 활용

In [26]:
# index=False 옵션은 DataFrame의 인덱스를 CSV 파일에 저장하지 않도록 합니다.
df_combined.to_csv("/content/drive/MyDrive/top_students_data.csv", index=False)

In [ ]:
# 나중에 파일 불러오고 싶으면
import pandas as pd
df_loaded = pd.read_csv("/content/drive/MyDrive/top_students_data.csv")
print(df_loaded.head())

결론

청소년 고민 순위 통계 데이터를 위와 같이 처리하여 매핑 딕셔너리를 생성하면,

상담 데이터의 자동 라벨링 시 객관적인 기준(중요도)을 반영할 수 있고,

층화 샘플링이나 데이터 증강 시에도 보다 균형 잡힌 데이터 분포를 유지할 수 있습니다.

이 방법을 적용하면 모델 파인튜닝이나 RAG 구성 시, 각 고민 주제에 따른 균형 잡힌 학습이 가능해져 전반적인 응답 품질이 향상될 것

## 각 csv  파일 읽고, prompt/response 열로 이름 통일

In [8]:
import pandas as pd

# PDF 데이터 (270건)
pdf_file = "/content/gpt_pdf_response_cleaned (1).csv"
df_pdf = pd.read_csv(pdf_file)

# CounselGPT 데이터 (6,500건)
counsel_file = "/content/select_students.csv"
df_counsel = pd.read_csv(counsel_file)

# 두 데이터셋의 열 확인
print("PDF 데이터 열:", df_pdf.columns)
print("CounselGPT 데이터 열:", df_counsel.columns)

# 열 이름을 통일하는 함수 (예: 'question' → 'prompt', 'answer' → 'response')
def standardize_columns(df):
    df.columns = df.columns.str.lower()  # 모두 소문자로 통일
    if "question" in df.columns:
        df = df.rename(columns={"question": "prompt"})
    if "answer" in df.columns:
        df = df.rename(columns={"answer": "response"})
    return df

df_pdf = standardize_columns(df_pdf)
df_counsel = standardize_columns(df_counsel)

PDF 데이터 열: Index(['prompt', 'response'], dtype='object')
CounselGPT 데이터 열: Index(['Unnamed: 0', 'prompt', 'response'], dtype='object')


## 기본 category 열 추가
- 각 데이터셋이 어디에서 왔는지 구분

In [9]:
# PDF 데이터에 category 열 추가 (270건 데이터)
if "category" not in df_pdf.columns:
    df_pdf["category"] = "모범 상담"

# CounselGPT 데이터에 category 열 추가 (6,500건 데이터)
if "category" not in df_counsel.columns:
    df_counsel["category"] = "청소년 상담"

In [21]:
import pandas as pd

# 1. CSV 파일 로드
pdf_file = "/content/gpt_pdf_response_cleaned (1).csv"  # PDF 데이터 (270건)
counsel_file = "/content/select_students.csv"            # CounselGPT 데이터 (6,500건)

df_pdf = pd.read_csv(pdf_file)
df_counsel = pd.read_csv(counsel_file)

# 2. 열 이름 표준화: 모든 열 이름을 소문자로 변환하고 'question'/'answer'가 있으면 'prompt'/'response'로 변경
def standardize_columns(df):
    df.columns = df.columns.str.lower()  # 모두 소문자로 변환
    if "question" in df.columns:
        df = df.rename(columns={"question": "prompt"})
    if "answer" in df.columns:
        df = df.rename(columns={"answer": "response"})
    return df

df_pdf = standardize_columns(df_pdf)
df_counsel = standardize_columns(df_counsel)

# 3. 기본 "category" 열 추가
# PDF 데이터는 '모범 상담', CounselGPT 데이터는 '청소년 상담'으로 지정
if "category" not in df_pdf.columns:
    df_pdf["category"] = "모범 상담"

if "category" not in df_counsel.columns:
    df_counsel["category"] = "청소년 상담"

# 4. 데이터셋 통합
df_combined = pd.concat([df_pdf, df_counsel], ignore_index=True)

# 5. 불필요한 열 제거: 'unnamed: 0' 같은 인덱스 열 삭제
df_combined = df_combined.loc[:, ~df_combined.columns.str.contains('^unnamed', case=False)]

# 6. 결측치 처리:
#    - 'prompt'와 'response' 컬럼에 결측치가 있으면 해당 행 제거
#    - 'category' 컬럼에 결측치가 있으면 '기타'로 채움
df_combined.dropna(subset=['prompt', 'response'], inplace=True)
df_combined['category'] = df_combined['category'].fillna('기타')

# 7. 결과 확인
print("통합된 데이터셋 크기:", df_combined.shape)
print(df_combined)

# 8. 통합된 데이터셋 저장 (선택사항)
output_file = "/content/drive/MyDrive/combined_counseling_data.csv"
df_combined.to_csv(output_file, index=False)
print("통합된 데이터셋이 저장되었습니다:", output_file)

통합된 데이터셋 크기: (6955, 3)
                                                 prompt  \
0     내담자: 저는 인문계 고등학교에 다니는 1 학년 여학생입니다 얼마전 학교에 서 문 ...   
1     내담자: 전 지금 고등학교 1 학년입니다 이제 2 학년이 되면 계열을 선택해야 하는...   
2     내담자: 저는 고 2 남학생입니다 곧 고 3 이 될텐데 집안 사정이 그리 좋지 못 ...   
3     내담자: 저는 고등학교 2 학년에 재학중인 여학생입니다 요즘 저는 제 장래희망 으로...   
4     내담자: 인문계 고등학교에 입학했는데 우리나라에서는 대학진학하는 것이 어려 울 것 ...   
...                                                 ...   
6950  내담자: 뭐.. 사람들이 저를 따르는 것을 좋아하더라고요. 좀 더 말하면, 친구들이...   
6951  내담자: 일을 너무 오래 안 한 탓인 것 같아요. 회사에서 일을 제대로 하지 못하면...   
6952  내담자: 저 얼마 전 친구가 죽었는데요. 그 때 제가 한 말이 친구의 불행한 사고에...   
6953  내담자: 그때 제가 했던 말이 친구의 죽음에 이어진다는 생각이 들어요. 그때 내가 ...   
6954  내담자: 지난학기까지는 잘 버티고 있었는데, 이번학기가 너무 힘들어요. 졸업하려면 ...   

                                               response category  
0     상담사: 대학입시로의 초기 관문인 계열선택을 놓고 나름대로 고민하며 상담실에 도움 ...    모범 상담  
1     상담사: 진로 문제로 편지를 주셨군요 문과계를 가야할지 자연계를 가야 할지 당신에 ...    모범 상담  
2     상담사: 원하는 대학에 문의하여 어떤 종류의 장학금 제도가 있는지 알아보도록 하십시...    모범 상담  


In [22]:
import pandas as pd

df_saved = pd.read_csv("/content/drive/MyDrive/combined_counseling_data.csv")
print(df_saved.shape)
print(df_saved.head())

(6955, 3)
                                              prompt  \
0  내담자: 저는 인문계 고등학교에 다니는 1 학년 여학생입니다 얼마전 학교에 서 문 ...   
1  내담자: 전 지금 고등학교 1 학년입니다 이제 2 학년이 되면 계열을 선택해야 하는...   
2  내담자: 저는 고 2 남학생입니다 곧 고 3 이 될텐데 집안 사정이 그리 좋지 못 ...   
3  내담자: 저는 고등학교 2 학년에 재학중인 여학생입니다 요즘 저는 제 장래희망 으로...   
4  내담자: 인문계 고등학교에 입학했는데 우리나라에서는 대학진학하는 것이 어려 울 것 ...   

                                            response category  
0  상담사: 대학입시로의 초기 관문인 계열선택을 놓고 나름대로 고민하며 상담실에 도움 ...    모범 상담  
1  상담사: 진로 문제로 편지를 주셨군요 문과계를 가야할지 자연계를 가야 할지 당신에 ...    모범 상담  
2  상담사: 원하는 대학에 문의하여 어떤 종류의 장학금 제도가 있는지 알아보도록 하십시...    모범 상담  
3  상담사: 종합예술인 연극무대를 많은 사람들이 동경하고 한 번쯤 해 보고 싶어 들 하...    모범 상담  
4  상담사: 최근 해외유학 자율화 방안이 시행된 후 중고생들 사이에서 조기유학에 대한 ...    모범 상담  


### 1. 데이터 분할

층화 샘플링을 통해 훈련, 검증, 테스트셋을 구성합니다.

### 2. 모델 선택 및 파인튜닝

한국어 특화 LLM(예: KoGPT2, KoAlpaca 등)을 선택하고, LoRA/PEFT 기법을 활용해 청소년 상담 데이터로 미세 조정합니다.

프롬프트 템플릿을 통해 상담사의 공감, 조언, 추가 질문 지시를 명확하게 설정합니다.

### 3. RAG 파이프라인 구축

FAISS와 HuggingFaceEmbeddings를 사용해 상담 데이터의 벡터 DB를 구축하고, LangChain의 RetrievalQA 체인을 구성하여 사용자 질문에 맞는 관련 사례를 검색해 응답 생성에 활용합니다.

### 4. 평가 및 지속적 개선

모델 성능을 정량적/정성적 평가하고, 사용자 피드백을 반영해 모델과 프롬프트, 데이터 증강, 샘플링 등을 반복 개선합니다.

## 1.데이터 분할 (층화 샘플링 적용)
통합된 데이터셋(6955행, 3열 – prompt, response, category)에서 각 카테고리("모범 상담", "청소년 상담")의 분포를 고려하여, 훈련/검증/테스트셋을 층화 샘플링으로 나눕니다.

In [27]:
import pandas as pd
from sklearn.model_selection import train_test_split

# 통합 데이터셋 불러오기
df = pd.read_csv("/content/drive/MyDrive/combined_counseling_data.csv")

# 카테고리 분포 확인
print("전체 카테고리 분포:")
print(df["category"].value_counts())

# 80%:20%로 먼저 훈련과 임시셋으로 분할 (stratify 적용)
train_set, temp_set = train_test_split(df, test_size=0.2, random_state=42, stratify=df["category"])

# 임시셋을 50:50으로 분할해서 검증과 테스트셋 구성 (각각 10%씩)
val_set, test_set = train_test_split(temp_set, test_size=0.5, random_state=42, stratify=temp_set["category"])

print("훈련 데이터셋 크기:", train_set.shape)
print("검증 데이터셋 크기:", val_set.shape)
print("테스트 데이터셋 크기:", test_set.shape)

전체 카테고리 분포:
category
청소년 상담    6683
모범 상담      272
Name: count, dtype: int64
훈련 데이터셋 크기: (5564, 3)
검증 데이터셋 크기: (695, 3)
테스트 데이터셋 크기: (696, 3)


## 2. 모델 선택 및 파인튜닝(Lora/PEFT)
2.1 모델 선택 - beomi/KoAlpaca-Polyglot-5.8B
2.2 Lora/PEFT 기법을 이용한 파인튜닝
- 로라나 펩트를 사용하면 기존 모델 전체를 재학습하지 않고, 적은 파라미터만 업데이트하여 도메인 특화 파인튜닝이 가능함

In [1]:
!pip install datasets transformers peft accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 92.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 93.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [30]:
# wandb 내 토큰
# ebf2ca4b961081c323dc30c6afdff7e9e7e91053

## training loss만 있는 코드

In [2]:
import pandas as pd
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    DataCollatorForSeq2Seq
)
from peft import LoraConfig, get_peft_model
import torch

# 1. 데이터셋 로드 및 전처리
data_path = "/content/drive/MyDrive/combined_counseling_data.csv"  # 통합된 데이터셋 경로
df = pd.read_csv(data_path)

# 우리가 사용할 컬럼: prompt, response
# 챗봇 파인튜닝을 위해 입력은 "내담자: {prompt}\n상담사:" 형태, 정답은 response
def format_example(example):
    input_text = f"{example['prompt']}\n상담사:"  # 입력 텍스트
    target_text = example["response"]             # 정답 텍스트
    return {"input_text": input_text, "target_text": target_text}

# DataFrame을 HuggingFace Dataset 객체로 변환 (필요한 컬럼만 선택)
dataset = Dataset.from_pandas(df[['prompt', 'response']])
dataset = dataset.map(format_example)

# 2. 모델 및 토크나이저 로드
model_name = "skt/kogpt2-base-v2"  # 한국어 특화 모델
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# 3. LoRA 설정 (PEFT 적용)
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["c_attn"],  # 모델 구조에 맞게 설정; 필요에 따라 조정
    lora_dropout=0.1,
    bias="none"
)
model = get_peft_model(model, lora_config)

# 4. 패딩 토큰 설정: pad_token이 없다면 [PAD]를 추가하고, 모델 임베딩 크기를 재조정합니다.
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))

# 5. 토크나이즈 및 전처리 함수 (padding 옵션 추가)
max_length = 1024  # 답변이 잘리지 않도록 충분히 긴 길이로 설정

def tokenize_function(example):
    # 입력 문장 토크나이즈 (고정 길이로 패딩 및 truncation)
    model_inputs = tokenizer(
        example["input_text"],
        max_length=max_length,
        truncation=True,
        padding="max_length"
    )
    # 정답 문장 토크나이즈 (동일한 방식 적용)
    labels = tokenizer(
        example["target_text"],
        max_length=max_length,
        truncation=True,
        padding="max_length"
    )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# batched=True로 토크나이즈 적용
tokenized_dataset = dataset.map(tokenize_function, batched=True)

# 6. 데이터 collator 설정
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# 7. TrainingArguments 설정
training_args = TrainingArguments(
    output_dir="./kogpt2_lora_finetuned",
    num_train_epochs=3,                    # 에포크: 3 (데이터 상황에 따라 조정)
    per_device_train_batch_size=8,         # 배치 사이즈: 8
    learning_rate=5e-5,
    warmup_steps=100,
    weight_decay=0.01,
    logging_steps=50,
    save_steps=500,
    evaluation_strategy="no",              # 평가 전략 미적용 (추후 검증셋 사용 가능)
    fp16=torch.cuda.is_available(),        # GPU가 있다면 fp16 사용
    report_to=[]                          # wandb 로그 기록 비활성화 (원하는 경우)
)

# 8. Trainer 설정 및 모델 파인튜닝
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

# 9. 파인튜닝된 모델 저장
output_model_dir = "./kogpt2_lora_finetuned_final"
model.save_pretrained(output_model_dir)
tokenizer.save_pretrained(output_model_dir)
print("파인튜닝된 모델이 저장되었습니다:", output_model_dir)


Map:   0%|          | 0/6955 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/513M [00:00<?, ?B/s]

...

Map:   0%|          | 0/6955 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/513M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-2-0d7c6e91b6de>:91: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModel`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss


KeyboardInterrupt: 

## training loss, validation loss 둘다 확인 가능 (skt/kogpt2-base-v2)

In [ ]:
import pandas as pd
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    DataCollatorForSeq2Seq
)
from peft import LoraConfig, get_peft_model
import torch

# 1. 데이터셋 로드 및 전처리
data_path = "/content/drive/MyDrive/combined_counseling_data.csv"  # 통합된 데이터셋 경로
df = pd.read_csv(data_path)

# 사용할 컬럼: prompt, response
# 챗봇 파인튜닝을 위해 입력은 "내담자: {prompt}\n상담사:" 형태, 정답은 response
def format_example(example):
    input_text = f"{example['prompt']}\n상담사:"  # 입력 텍스트
    target_text = example["response"]             # 정답 텍스트
    return {"input_text": input_text, "target_text": target_text}

# DataFrame을 HuggingFace Dataset 객체로 변환 (필요한 컬럼만 선택)
dataset = Dataset.from_pandas(df[['prompt', 'response']])
dataset = dataset.map(format_example)

# 2. 모델 및 토크나이저 로드
model_name = "skt/kogpt2-base-v2"  # 한국어 특화 모델, gpu 가능하면 beomi/KoAlpaca-Polyglot-5.8B or HuggingFaceH4/zephyr-7b-alpha(외국 모델이라 영어로도 대답함)
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# 3. LoRA 설정 (PEFT 적용)
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["c_attn"],  # 모델 구조에 맞게 설정; 필요에 따라 조정
    lora_dropout=0.1,
    bias="none"
)
model = get_peft_model(model, lora_config)

# 4. 패딩 토큰 설정: pad_token이 없다면 [PAD]를 추가하고, 모델 임베딩 크기를 재조정합니다.
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))

# 5. 토크나이즈 및 전처리 함수 (padding 옵션 추가)
max_length = 1024  # 답변이 잘리지 않도록 충분히 긴 길이로 설정

def tokenize_function(example):
    # 입력 문장 토크나이즈 (고정 길이 패딩 및 truncation 적용)
    model_inputs = tokenizer(
        example["input_text"],
        max_length=max_length,
        truncation=True,
        padding="max_length"
    )
    # 정답 문장 토크나이즈 (동일한 방식 적용)
    labels = tokenizer(
        example["target_text"],
        max_length=max_length,
        truncation=True,
        padding="max_length"
    )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# batched=True로 토크나이즈 적용
tokenized_dataset = dataset.map(tokenize_function, batched=True)

# 5-1. 데이터셋 분할: 전체 데이터셋을 train(90%) / eval(10%)로 분할
split_dataset = tokenized_dataset.train_test_split(test_size=0.1, seed=42)
train_dataset = split_dataset["train"]
eval_dataset = split_dataset["test"]

# 6. 데이터 collator 설정
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# 7. TrainingArguments 설정 (평가 전략 포함)
training_args = TrainingArguments(
    output_dir="./kogpt2_lora_finetuned",
    num_train_epochs=3,                    # 에포크: 3 (데이터에 따라 조정)
    per_device_train_batch_size=8,         # 배치 사이즈: 8
    learning_rate=5e-5,
    warmup_steps=100,
    weight_decay=0.01,
    logging_steps=50,
    save_steps=500,
    save_strategy="epoch",                 # 평가 및 저장 전략을 동일하게 "epoch"으로 설정
    evaluation_strategy="epoch",            # 매 에포크마다 평가
    load_best_model_at_end=True,            # 최고의 모델 저장
    metric_for_best_model="eval_loss",      # 평가 지표: 검증 손실
    greater_is_better=False,                # 낮은 loss가 더 좋음
    fp16=torch.cuda.is_available(),        # GPU가 있다면 fp16 사용
    report_to=[]                          # wandb 로그 기록 비활성화 (원하는 경우)
)

# 8. Trainer 설정 및 모델 파인튜닝
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,             # 검증 데이터셋 추가
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

# 9. 파인튜닝된 모델 저장
output_model_dir = "/content/drive/MyDrive/kogpt2_lora_finetuned_final"
model.save_pretrained(output_model_dir)
tokenizer.save_pretrained(output_model_dir)
print("파인튜닝된 모델이 저장되었습니다:", output_model_dir)

Map:   0%|          | 0/6955 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/layer.py:1264: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


Map:   0%|          | 0/6955 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-3-44fe7cee5b4a>:100: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModel`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Epoch,Training Loss,Validation Loss


## (beomi/KoAlpaca-Polyglot-5.8B) 사용 코드 - gpu에서 가능할듯

In [ ]:
import pandas as pd
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    DataCollatorForSeq2Seq
)
from peft import LoraConfig, get_peft_model
import torch

# 1. 데이터셋 로드 및 전처리
data_path = "/content/drive/MyDrive/combined_counseling_data.csv"  # 통합된 데이터셋 경로
df = pd.read_csv(data_path)

# 사용할 컬럼: prompt, response
# 챗봇 파인튜닝을 위해 입력은 "내담자: {prompt}\n상담사:" 형태, 정답은 response
def format_example(example):
    input_text = f"{example['prompt']}\n상담사:"  # 입력 텍스트
    target_text = example["response"]             # 정답 텍스트
    return {"input_text": input_text, "target_text": target_text}

# DataFrame을 HuggingFace Dataset 객체로 변환 (필요한 컬럼만 선택)
dataset = Dataset.from_pandas(df[['prompt', 'response']])
dataset = dataset.map(format_example)

# 2. 모델 및 토크나이저 로드
# 모델 이름을 beomi/KoAlpaca-Polyglot-5.8B로 변경 (GPU 환경에서 사용 권장)
model_name = "beomi/KoAlpaca-Polyglot-5.8B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# 3. LoRA 설정 (PEFT 적용)
# 주의: target_modules 설정은 모델 구조에 따라 달라질 수 있음. 여기서는 기본적으로 "c_attn"을 사용하도록 하였습니다.
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["c_attn"],  # KoAlpaca 모델의 구조에 맞게 필요시 수정하세요.
    lora_dropout=0.1,
    bias="none"
)
model = get_peft_model(model, lora_config)

# 4. 패딩 토큰 설정: pad_token이 없다면 [PAD]를 추가하고, 모델 임베딩 크기를 재조정합니다.
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))

# 5. 토크나이즈 및 전처리 함수 (padding 옵션 추가)
max_length = 1024  # 답변이 잘리지 않도록 충분히 긴 길이로 설정

def tokenize_function(example):
    # 입력 문장 토크나이즈 (고정 길이 패딩 및 truncation 적용)
    model_inputs = tokenizer(
        example["input_text"],
        max_length=max_length,
        truncation=True,
        padding="max_length"
    )
    # 정답 문장 토크나이즈 (동일한 방식 적용)
    labels = tokenizer(
        example["target_text"],
        max_length=max_length,
        truncation=True,
        padding="max_length"
    )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# batched=True로 토크나이즈 적용
tokenized_dataset = dataset.map(tokenize_function, batched=True)

# 5-1. 데이터셋 분할: 전체 데이터셋을 train(90%) / eval(10%)로 분할
split_dataset = tokenized_dataset.train_test_split(test_size=0.1, seed=42)
train_dataset = split_dataset["train"]
eval_dataset = split_dataset["test"]

# 6. 데이터 collator 설정
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# 7. TrainingArguments 설정 (평가 전략 포함)
training_args = TrainingArguments(
    output_dir="./beomi_lora_finetuned",   # 학습 체크포인트 및 로그 경로
    num_train_epochs=3,                        # 에포크: 3 (데이터에 따라 조정)
    per_device_train_batch_size=8,             # 배치 사이즈: 8
    learning_rate=5e-5,
    warmup_steps=100,
    weight_decay=0.01,
    logging_steps=50,
    save_strategy="epoch",                     # 평가 및 저장 전략: 매 에포크마다 저장
    evaluation_strategy="epoch",               # 매 에포크마다 평가
    load_best_model_at_end=True,               # 최고의 모델 저장
    metric_for_best_model="eval_loss",         # 평가 지표: 검증 손실
    greater_is_better=False,                   # 낮은 loss가 좋음
    fp16=torch.cuda.is_available(),            # GPU 있으면 fp16 사용
    report_to=[]                               # wandb 로그 기록 비활성화 (원하는 경우)
)

# 8. Trainer 설정 및 모델 파인튜닝
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

# 9. 파인튜닝된 모델 저장
output_model_dir = "/content/drive/MyDrive/beomi_lora_finetuned_final"
model.save_pretrained(output_model_dir)
tokenizer.save_pretrained(output_model_dir)
print("파인튜닝된 모델이 저장되었습니다:", output_model_dir)

## 3. RAG 파이프라인 구축

FAISS와 HuggingFaceEmbeddings를 이용해 상담 데이터 벡터 DB를 만들고, LangChain RetrievalQA 체인을 구성하여 관련 사례 검색 및 응답 생성

1. 통합 데이터셋(CSV 파일)을 불러와서 각 행의 prompt와 response를 결합하여 하나의 문서로 만듦

2. 문서들을 적절한 길이로 분할

3. HuggingFaceEmbeddings (한국어에 적합한 ko-sroberta-multitask 모델)를 사용해 FAISS 벡터 DB를 구축하고, 이를 Retriever로 변환

4. 상담사 역할을 명확히 하는 프롬프트 템플릿을 정의

5. 이미 파인튜닝한 모델(이전 단계에서 finetuned_model과 tokenizer가 준비됨)을 inference 모드로 wrapping하여 HuggingFacePipeline LLM으로 사용

6. LangChain의 RetrievalQA 체인을 구성하여, 사용자의 질문에 대해 관련 사례를 검색하고 응답을 생성

7. 테스트 질문을 넣어 결과(챗봇 답변과 사용된 소스 문서 일부)를 출력

In [ ]:
!pip install pandas datasets transformers langchain langchain_community peft faiss-cpu torch

In [ ]:
import pandas as pd
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from transformers import pipeline
from langchain.llms import HuggingFacePipeline

# 1. 통합 데이터셋 로드
data_path = "/content/drive/MyDrive/combined_counseling_data.csv"
df = pd.read_csv(data_path)
# 필요한 열만 사용 (예: prompt, response, category)
df = df[['prompt', 'response', 'category']]

# 2. 각 행의 prompt와 response를 결합하여 하나의 문서를 만듭니다.
def combine_text(row):
    # prompt와 response를 줄바꿈 문자로 결합하여 하나의 텍스트 문서 생성
    return {"text": row["prompt"] + "\n" + row["response"]}

documents = df.apply(combine_text, axis=1).tolist()

# 3. 문서 분할: 문서가 너무 길 경우 일정 길이(여기서는 500 토큰 단위)로 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_docs = text_splitter.split_documents(documents)

# 4. FAISS 벡터 DB 구축: 한국어에 적합한 임베딩 모델로 문서를 임베딩
embedding_model = HuggingFaceEmbeddings(model_name="jhgan/ko-sroberta-multitask")
vectordb = FAISS.from_documents(split_docs, embedding_model)
retriever = vectordb.as_retriever(search_kwargs={"k": 3})

# 5. 상담사 역할 프롬프트 템플릿 정의
template = """
너는 청소년 대상 심리상담사야.
내담자의 감정을 진심으로 공감하며 따뜻하게 위로해줘.
구체적인 해결책과 조언을 제시하고, 필요한 경우 추가 질문을 던져서 대화를 이어가.
질문: {question}
답변:
"""
prompt = PromptTemplate.from_template(template)

# 6. Finetuned 모델을 inference 용으로 래핑: 여기서는 이전 단계에서 파인튜닝된 model과 tokenizer를 사용합니다.
# model과 tokenizer는 이미 로드 및 파인튜닝 된 상태라고 가정
inference_pipe = pipeline(
    "text-generation",
    model=model,              # 파인튜닝된 모델(로라)이 여기에 들어감
    tokenizer=tokenizer,
    max_new_tokens=256,
    do_sample=False          # 일관성을 위해 샘플링 없이 생성
)
llm = HuggingFacePipeline(pipeline=inference_pipe)

# 7. RetrievalQA 체인 구성: retriever와 LLM, 프롬프트 템플릿을 결합
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt}
)

# 8. 테스트: 사용자 질문에 대해 챗봇 답변 생성 및 사용된 소스 문서 출력
# 여러 테스트 질문 목록 작성
test_questions = [
    "요즘 친구 관계 때문에 많이 고민해요. 어떻게 해야 할까요?",
    "학교 생활이 너무 힘들어요. 조언 부탁드립니다.",
    "인스타그램에서 친구들이 자랑하는 모습을 보면 자격지심이 들고, 제 생활이 부족하게 느껴져요. 어떻게 해야 할까요?"
    "가족과의 갈등이 심해요. 어떻게 해결할 수 있을까요?",
    "시험 성적이 계속 떨어져서 자존감이 낮아졌어요. 극복 방법이 있을까요?",
    "진로에 대해 고민이 많아요. 어떤 길이 제게 맞을지 조언해 주세요."
]

# 각 질문에 대해 RAG 체인을 호출하고 결과 출력
for question in test_questions:
    result = qa_chain.invoke(question)
    print("============================================")
    print("질문:")
    print(question)
    print("============================================")
    print("챗봇 답변:")
    print(result["result"])
    print("============================================")
    print("사용된 소스 문서 일부:")
    for doc in result["source_documents"]:
        # 소스 문서의 앞 200자 출력 (너무 길면 잘림)
        print(doc["text"][:200] + "...")
    print("\n\n")
